In [1]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.append(str(project_root))

In [2]:
import joblib
import numpy as np
from models.utils import get_artifacts_path
from db.supabase.queries_ohlcv import fetch_ohlcv
from db.utils_ohlcv import get_ibex_tickers
from models.trees.features import safe_build_features


model_type = "rf"
horizon = 1
name = f"{model_type}_h{horizon}_full.pkl"
artifact = joblib.load(get_artifacts_path() / name)
model = artifact["model"]
feature_cols = artifact["features"]


tickers = get_ibex_tickers()
df_micro = fetch_ohlcv(tickers, 50)
print(df_micro.shape)
df_micro.tail(1)

(1500, 7)


,ticker,date,open,high,low,close,volume
1499,UNI.MC,2026-02-19,2.708,2.72,2.682,2.694,4109302


In [5]:
df = safe_build_features(df_micro, horizon)
remove_cols = [
    "ticker", "date", "open", "high", "low",
    "close", "volume", "target", "future_log_ret"
]
X = df.drop(columns=remove_cols)
X = X[feature_cols]
X = X.replace([np.inf, -np.inf], np.nan)
mask = X.notna().all(axis=1)
X = X.loc[mask]

df_pred = df.loc[mask, ["ticker", "date"]]


In [7]:
preds = model.predict(X)
proba = model.predict_proba(X)[:, 1]

df_pred["pred"] = preds
df_pred["proba"] = proba

# double check only one prediction for ticker
df_pred = df_pred.sort_values("date").groupby("ticker").tail(1)


# select 3 highest confidence (deviation from 0.5)
df_pred["confidence"] = abs(df_pred["proba"] - 0.5)
df_last = df_pred.sort_values("confidence", ascending=False).head(3).reset_index()[["ticker", "pred", "proba"]]
df_last


,ticker,pred,proba
0,BKT.MC,1,0.511793
1,MTS.MC,0,0.488899
2,UNI.MC,1,0.511091


In [8]:
df_pred

,ticker,date,pred,proba,confidence
49,ACS.MC,2026-02-19,1,0.501912,0.001912
1399,SAN.MC,2026-02-19,1,0.506174,0.006174
1349,SAB.MC,2026-02-19,1,0.504661,0.004661
1299,RED.MC,2026-02-19,1,0.505969,0.005969
1249,PUIG.MC,2026-02-19,1,0.507517,0.007517
1199,NTGY.MC,2026-02-19,1,0.503999,0.003999
1149,MTS.MC,2026-02-19,0,0.488899,0.011101
1099,MRL.MC,2026-02-19,1,0.506746,0.006746
1049,MAP.MC,2026-02-19,1,0.506409,0.006409
999,LOG.MC,2026-02-19,0,0.499163,0.000837
